<a href="https://colab.research.google.com/github/Fejcvk/data-mining/blob/polbert-experiment/src/notebookes/bert_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

For now we need just pandas for parsing dataset and transformers for BERT

In [1]:
!pip install transformers
import pandas as pd
from transformers import *
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter


Read excel file, skip headers and 1st row with column nasmes

PS : Small hack with deleting biggest files from dataset. Gonna be fixed after obtaining bigger resources

In [2]:
df= pd.read_excel('dataset.xlsx', header=None, skiprows=1)
df.drop(df[df[4].map(len) > 160 ].index, inplace=True)
df.reset_index()

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,0,1203,"I said, ""I have a test, I'm gonna fail biology...",1,To which I said that I was to write a class te...,"Ja na to, że mam klasówkę, że zawalę biologię ...",N,eliptyczne,2531,mówienia,NaN,NaN,1,nie dotyczy,nie ma hasła,[elipsa],o?,?,NaN,mam klasówkę,brak,na to,0,future,nie dotyczy,NaN,NaN,NaN,NaN,N,E,?,N,NaN
1,1,1671,Gliński did not have to turn his head to guess...,1,Gliński did not have to turn his head to guess...,Ani że na progu stanął właśnie Romanyczko.,?,eliptyczne,2978,epistemiczny,NaN,NaN,1,?,nie ma hasła,[elipsa],?,?,"Gliński nie musiał odwracać głowy, by nie domy...",na progu stanął właśnie Romanyczko.,past,"nie musiał odwracać głowy, by domyślić się,",0,past,no,E,E,E,E,NaN,NaN,NaN,NaN,NaN
2,2,2441,Then he'll bring the Chinese back.,1,"Earlier Ukrainians were said to come and work,...","Potem, że się przywiezie Chińczyków.",N,eliptyczne,3489,mówienia,NaN,NaN,1,nie dotyczy,nie ma hasła,[elipsa],?,?,NaN,się przywiezie Chińczyków.,past,elipsa (mówiło się),0,future,nie dotyczy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,945,He got weaker and weaker and felt that it was ...,0,He got weaker and weaker and felt that it was ...,"Słabł coraz bardziej, czuł, że to kwestia zale...",N,1,2119,epistemiczny,percepcyjny,NaN,2,nie dotyczy,http://plwordnet.pwr.wroc.pl/wordnet/b8d08aa2-...,"[e] czuć, że_","""+""","""+""","Słabł coraz bardziej, nie czuł, że to kwestia ...",to kwestia zaledwie minut.,past,czuł,0,inne,nie dotyczy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,947,"However, people who are very seriously ill, wh...",0,"However, people who are very seriously ill, wh...","Jednak ludzie bardzo ciężko chorzy, gdy już cz...",N,1,2123,epistemiczny,percepcyjny,NaN,2,nie dotyczy,http://plwordnet.pwr.wroc.pl/wordnet/b8d08aa2-...,"[e] czuć, że_",o,o,NaN,medycyna też nie może im już pomóc,present,czują,0,present,nie dotyczy,NaN,NaN,NaN,NaN,N,N,E,E,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,2590,1004,The police immediately established that the ca...,1,The police immediately found out the car to be...,"Policjanci natychmiast ustalili, że samochód n...",N,1,2377,wnioskowania,epistemiczny,NaN,410,nie dotyczy,http://plwordnet.pwr.wroc.pl/wordnet/bc2c6ba8-...,"ustalić [na podstawie czego], że_",o,o,NaN,samochód należy do jednego ze szczecińskich pr...,past,ustalili,0,present,nie dotyczy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1975,2591,1045,It was established that both girls could stay ...,1,It has been established that both girls accomp...,"Udało się ustalić, że obie dziewczynki mogą pr...",N,1,2407,wnioskowania,epistemiczny,NaN,410,nie dotyczy,http://plwordnet.pwr.wroc.pl/wordnet/bc2c6ba8-...,"ustalić [na podstawie czego], że_",o,o,NaN,obie dziewczynki mogą przebywać w mieszkaniu p...,past,Udało się ustalić,0,present,nie dotyczy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976,2592,2258,It was established that he had close social re...,1,It was established that he had close social re...,"Ustalono, że łączą go z niektórymi członkami g...",N,1,3369,wnioskowania,epistemiczny,NaN,410,nie dotyczy,http://plwordnet.pwr.wroc.pl/wordnet/bc2c6ba8-...,"ustalić [na podstawie czego], że_",o,o,NaN,łączą go z niektórymi członkami grupy bliskie...,past,ustalono,0,present,nie dotyczy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1977,2594,443,Of course you can love if you see that this lo...,1,Of course you can love someone if you see that...,"Oczywiście, że można kochać, jeśli zobaczy się...",N,warunkowe,436,epistemiczny,NaN,NaN,411,nie dotyczy,http://plwordnet.pwr.wroc.pl/wordnet/b9e36504-...,"[e] zobaczyć, że_","""+""","""+""",NaN,ona miała do niej prawo.,future,zobaczy się,0,past,nie dotyczy,NaN,NaN,NaN,NaN,N,E,E,E,NaN


For our smaller version of dataset just take two features that we are interested in, namely T PL and GOLD<T,H>

In [0]:
dataset = df[[4, 5]]

In [4]:
dataset.head

<bound method NDFrame.head of                                                       4  5
0     Ja na to, że mam klasówkę, że zawalę biologię ...  N
1            Ani że na progu stanął właśnie Romanyczko.  ?
2                  Potem, że się przywiezie Chińczyków.  N
3     Słabł coraz bardziej, czuł, że to kwestia zale...  N
4     Jednak ludzie bardzo ciężko chorzy, gdy już cz...  N
...                                                 ... ..
2590  Policjanci natychmiast ustalili, że samochód n...  N
2591  Udało się ustalić, że obie dziewczynki mogą pr...  N
2592  Ustalono, że łączą go z niektórymi członkami g...  N
2594  Oczywiście, że można kochać, jeśli zobaczy się...  N
2595  Oczywiście, że można kochać, jeśli zobaczy się...  N

[1979 rows x 2 columns]>

Change labels of ECN for numeric values

In [0]:
labels = {'E':0,'C':1,'N':2,'?':3}

for idx, row in dataset.iterrows():
    row[5] = labels[row[5]]

In [6]:
dataset

,4,5
0,"Ja na to, że mam klasówkę, że zawalę biologię ...",2
1,Ani że na progu stanął właśnie Romanyczko.,3
2,"Potem, że się przywiezie Chińczyków.",2
3,"Słabł coraz bardziej, czuł, że to kwestia zale...",2
4,"Jednak ludzie bardzo ciężko chorzy, gdy już cz...",2
...,...,...
2590,"Policjanci natychmiast ustalili, że samochód n...",2
2591,"Udało się ustalić, że obie dziewczynki mogą pr...",2
2592,"Ustalono, że łączą go z niektórymi członkami g...",2
2594,"Oczywiście, że można kochać, jeśli zobaczy się...",2


In [7]:
dataset[5].value_counts()

2    1214
0     634
3      73
1      58
Name: 5, dtype: int64

In [8]:
max_len = 0
m_idx = 0
for idx,row in dataset.iterrows():
    l = len(row[4])
    if l > max_len:
        max_len = l
        m_idx = idx
print(m_idx)

37


In [9]:
print(dataset[4][m_idx])

Za tę moją słabość stokrotnie Cię przepraszam, ale musisz już brać pod uwagę, że jestem stary, zmęczony, przepracowany, chory i nie mam już tyle sił co dawniej.


In [10]:
!pip install sklearn
from sklearn.model_selection import train_test_split

# Polbert

Here we import pretrained version of Bert for Polish language

In [0]:
model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

## Tokenization

Tokenize every sentence from our dataset

In [0]:
tokenized = dataset[4].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [13]:
tokenized.values

array([list([2, 2055, 1898, 1907, 16, 2095, 2040, 15701, 1889, 3118, 16, 2095, 32091, 1014, 12071, 1014, 51, 1893, 15887, 1012, 1906, 27508, 2165, 4785, 18, 4]),
       list([2, 2937, 2095, 1898, 26468, 40126, 1019, 29480, 6977, 1013, 7150, 18, 4]),
       list([2, 3006, 16, 2095, 2243, 11319, 2713, 11007, 8572, 1889, 18, 4]),
       ...,
       list([2, 21075, 16, 2095, 34538, 2126, 2015, 68, 3694, 2818, 1998, 20444, 5233, 26559, 17765, 39327, 51, 2095, 4384, 1937, 12294, 2047, 38360, 2713, 1910, 68, 43080, 9916, 5134, 12522, 3942, 18, 4]),
       list([2, 31040, 16, 2095, 29801, 6289, 1015, 16, 10865, 3403, 2243, 16, 2095, 2259, 16879, 1015, 1893, 40065, 1923, 2243, 68, 4109, 16, 2095, 2499, 3089, 1906, 3115, 3410, 18, 4]),
       list([2, 31040, 16, 2095, 29801, 6289, 1015, 16, 10865, 3403, 2243, 16, 2095, 2259, 16879, 1015, 1893, 40065, 1923, 2243, 68, 4109, 16, 2095, 2499, 3089, 1906, 3115, 3410, 18, 4])],
      dtype=object)

## Padding

We need to have input of the same lenght in order to feed BERT once (we will get performance speed). This will be some simple python string manipulations

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

In [15]:
max_len

47

Maybe we can consider shortening some sentences here

In [0]:
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [17]:
padded

array([[    2,  2055,  1898, ...,     0,     0,     0],
       [    2,  2937,  2095, ...,     0,     0,     0],
       [    2,  3006,    16, ...,     0,     0,     0],
       ...,
       [    2, 21075,    16, ...,     0,     0,     0],
       [    2, 31040,    16, ...,     0,     0,     0],
       [    2, 31040,    16, ...,     0,     0,     0]])

In [18]:
np.array(padded).shape

(1979, 47)

## Computing embedding using BERT

Now we do create cube of 146(tokens per row) * 768 (hidden layers) * 2596

In [0]:
input_ids = torch.tensor(np.array(padded))

In [20]:
input_ids

tensor([[    2,  2055,  1898,  ...,     0,     0,     0],
        [    2,  2937,  2095,  ...,     0,     0,     0],
        [    2,  3006,    16,  ...,     0,     0,     0],
        ...,
        [    2, 21075,    16,  ...,     0,     0,     0],
        [    2, 31040,    16,  ...,     0,     0,     0],
        [    2, 31040,    16,  ...,     0,     0,     0]])

In [0]:
with torch.no_grad():
    last_hidden_states = model(input_ids)

We will extract two sets of embeddings. One for CSL only (2D array) and one for the whole cube

In [0]:
features = last_hidden_states[0][:,:,:]

In [0]:
cls_features = last_hidden_states[0][:,0,:]

In [25]:
features.shape

torch.Size([1979, 47, 60000])

In [26]:
cls_features.shape

torch.Size([1979, 60000])

In [0]:
y_labes = np.array(dataset[5], dtype=np.int64)
y_labes.shape
y_labes = torch.from_numpy(y_labes)

## Feed Forward Neural Network
Creating Feed Forward Neural Network which will take as an input vector of embeddings and return one of semantic classes. Firstly we will do it for 2d CLS matrix

In [0]:
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as ds

### Creating dataset

In [1]:
train_features, test_features, train_labels, test_labels = train_test_split(cls_features, y_labes)
print(train_labels)

NameError: ignored

In [0]:
batch_size = int(train_features.shape[0] / 100)
n_iters = batch_size * 100
epochs = int(n_iters / (len(train_features) / batch_size))

In [0]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out

In [110]:
input_dim = 60000
hidden_dim = 100
output_dim = len(labels)

input_dim

60000

In [0]:
ffnn = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [0]:
loss_fn = nn.BCELoss()

In [0]:
learning_rate = 0.1
optimizer = torch.optim.SGD(ffnn.parameters(), lr=learning_rate) 

In [0]:
def generate_batch_data(x, y, batch_size):
    i, batch = 0, 0
    for batch, i in enumerate(range(0, len(x) - batch_size, batch_size), 1):
        x_batch = x[i : i + batch_size]
        y_batch = y[i : i + batch_size]
        yield x_batch, y_batch, batch
    if i + batch_size < len(x):
        yield x[i + batch_size :], y[i + batch_size :], batch + 1
    if batch == 0:
        yield x, y, 1

In [116]:
for epoch in range(0,epochs):
  ffnn.train(True)
  train_loss = 0 
  for x_batch, y_batch, batch in generate_batch_data(train_features, train_labels, batch_size):
    y_pred = ffnn(x_batch)
    print(y_pred)
    print(y_batch)
    optimizer.zero_grad()
    loss = loss_fn(y_pred, y_batch)

    # Getting gradients w.r.t. parameters
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

tensor([[-0.0239,  0.8916,  0.7854,  1.7021],
        [-0.4758,  0.8637,  0.2449,  0.8873],
        [ 0.1192,  1.2056,  0.3283,  1.1945],
        [-0.2715,  0.9123,  0.4954,  0.9474],
        [-0.3159,  0.8381,  0.6775,  0.5324],
        [-0.3117,  0.7091,  0.4444,  0.6947],
        [-0.2734,  0.8192,  0.5182,  0.5749],
        [-0.3396,  0.9162,  0.5793,  0.8510],
        [-0.3934,  1.3569,  0.6973,  0.4621],
        [-0.2140,  0.9693,  0.5053,  1.2807],
        [-0.1567,  0.5827,  0.3734,  0.6932],
        [-0.3501,  0.8327,  0.5674,  0.7219],
        [-0.3085,  0.8453,  0.6104,  0.9911],
        [-0.1493,  0.7515,  0.5648,  0.8816]], grad_fn=<AddmmBackward>)
tensor([0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 1, 2, 0])


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([14])) that is different to the input size (torch.Size([14, 4])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


ValueError: ignored